# Dividing Video into 4 parts


In [ ]:
import cv2

# Replace 'your_video.mp4' with the path to your video file
video_path = r'd:\Capstone\Yoga Capstone\videos1\correct\1.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get the frames per second (fps) and total frame count
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the frame count for each segment
frames_per_segment = total_frames // 4

for i in range(4):
    # Calculate the start and end frames for the segment
    start_frame = i * frames_per_segment
    end_frame = (i + 1) * frames_per_segment if i < 3 else total_frames
    
    # Set the output file name
    output_file = f'C:/Users/Lenovo/Desktop/segment{i + 1}.mp4'
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))
    
    # Read and write frames for the segment
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    while cap.isOpened() and cap.get(cv2.CAP_PROP_POS_FRAMES) < end_frame:
        ret, frame = cap.read()
        if ret:
            out.write(frame)
        else:
            break
    
    # Release the video writer for this segment
    out.release()

# Release the video capture object
cap.release()


## Multi-Threading

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import threading

# Define your video processing function
def process_video(video_path, window_name, results):
    IMG_WIDTH, IMG_HEIGHT = 224, 224
    
    # Load the saved model
    model = tf.keras.models.load_model(r'd:\Capstone\koi bhi\Top_5_Models\pose_classification_model_mobilenetv2.h5')

    # Function to preprocess the frame for prediction
    def preprocess_frame(frame, img_width, img_height):
        resized_frame = cv2.resize(frame, (img_width, img_height))
        preprocessed_frame = np.expand_dims(resized_frame, axis=0)
        preprocessed_frame = preprocessed_frame / 255.0
        return preprocessed_frame

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Variables to keep track of correct, partially correct, and incorrect frames
    total_frames = 0
    correct_frames = 0
    partially_correct_frames = 0
    incorrect_frames = 0
    none_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame for prediction
        preprocessed_frame = preprocess_frame(frame, IMG_WIDTH, IMG_HEIGHT)

        # Make predictions on the frame
        predictions = model.predict(preprocessed_frame)
        predicted_class = np.argmax(predictions, axis=1)

        # Update counters based on the predicted class
        total_frames += 1
        if predicted_class[0] == 0:  # Correct class index
            correct_frames += 1
        elif predicted_class[0] == 3:  # Partially Correct class index
            partially_correct_frames += 1
        elif predicted_class[0] == 1:  # Incorrect class index
            incorrect_frames += 1
        elif predicted_class[0] == 2:  # None class index
            none_frames += 1

        # Get the class label as a string
        class_labels = ["Correct", "Incorrect", "None", "Partially Correct"]
        predicted_class_label = class_labels[predicted_class[0]]

        # Display the predicted class on the frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 2
        text_size = cv2.getTextSize(predicted_class_label, font, font_scale, font_thickness)[0]
        text_position = (10, 30)
        text_background_position = (10, 10)
        text_background_end_position = (10 + text_size[0] + 10, 10 + text_size[1] + 10)
        cv2.rectangle(frame, text_background_position, text_background_end_position, (0, 0, 255), cv2.FILLED)
        cv2.putText(frame, predicted_class_label, text_position, font, font_scale, (255, 255, 255), font_thickness)

        # Display the frame in a separate window
        cv2.imshow(window_name, frame)

        # Exit the loop when 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Calculate the scores
    step = 100 / total_frames
    score1 = step * correct_frames
    score2 = step / 2 * partially_correct_frames
    total_score = score1 + score2

    # Store the statistics in the results dictionary
    results[window_name] = {
        "Total Frames": total_frames,
        "Correct Frames": correct_frames,
        "Partially Correct Frames": partially_correct_frames,
        "Incorrect Frames": incorrect_frames,
        "None Frames": none_frames,
        "Total Score": total_score
    }

    # Release the video capture and close the display window
    cap.release()
    cv2.destroyWindow(window_name)

# List of video file paths and corresponding window names
video_files = [
    (r'c:\Users\Lenovo\Desktop\segment1.mp4', "Video 1"),
    (r'c:\Users\Lenovo\Desktop\segment2.mp4', "Video 2"),
    (r'c:\Users\Lenovo\Desktop\segment3.mp4', "Video 3"),
    (r'c:\Users\Lenovo\Desktop\segment4.mp4', "Video 4")
]

# Create a dictionary to store the results for each video
results = {}

# Create a thread for each video processing task
threads = []

for video_path, window_name in video_files:
    thread = threading.Thread(target=process_video, args=(video_path, window_name, results))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

# Calculate and display the overall scores
overall_total_frames = sum(stats["Total Frames"] for stats in results.values())
overall_correct_frames = sum(stats["Correct Frames"] for stats in results.values())
overall_partially_correct_frames = sum(stats["Partially Correct Frames"] for stats in results.values())
overall_incorrect_frames = sum(stats["Incorrect Frames"] for stats in results.values())
overall_none_frames = sum(stats["None Frames"] for stats in results.values())

step = 100 / overall_total_frames
overall_score1 = step * overall_correct_frames
overall_score2 = step / 2 * overall_partially_correct_frames
overall_total_score = overall_score1 + overall_score2

print("Overall Scores:")
print(f"Total Frames: {overall_total_frames}")
print(f"Correct Frames: {overall_correct_frames}")
print(f"Partially Correct Frames: {overall_partially_correct_frames}")
print(f"Incorrect Frames: {overall_incorrect_frames}")
print(f"None Frames: {overall_none_frames}")
print(f"Total Score: {overall_total_score:.2f}")

# print("Individual Video Scores:")
# for video_name, video_stats in results.items():
#     print(f"{video_name}:")
#     for stat_name, stat_value in video_stats.items():
#         print(f"{stat_name}: {stat_value:.2f}")

print("All videos processed.")
